# Credit Scoring App:
Case study :
The process of accepting a consumer credit must be very fast in
especially when customers require store credit at checkout.
purchase (example: payment in three installments free of charge, etc.).
For a very long time, banks and credit organizations have developed
automatic and instant methods of rating the creditworthiness of customers and
their ability to repay the loan. These tools are very useful for an acceptance
rapid credit granting, but also to determine the amount of the envelope
credit authorization or to monitor the risk of customers and
loan portfolios.
Credit score is a function that assigns a credit quality value to a
customer or a loan according to explanatory variables such as the debt ratio
of the borrower, his account behavior or any other quantity that is
correlated to borrower default.

### 1-set up the environment:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn  as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
%matplotlib inline
print("set up completed")